Created on 02/06/2024

Author - Ragavee Mekala

Description - This code creates reg files for all/each age bin of stellar clusters which can be opened in the DS9 software. Uncomment the required age bin.

In [ ]:
import pandas as pd
from astropy.table import Table

# read catalog
cluster_data = input('Enter the filename of the cluster data sheet: ') # example input : NGC5194_datasheet.csv
df = pd.read_csv(cluster_data)

# CSV to Astropy Table
table = Table.from_pandas(df)

# prepare .reg file
reg_filename = input('Enter the desired filename of the reg file according to the age bin: ') # example input : NGC5194_datasheet_9.7andabove.reg
with open(reg_filename, 'w') as regfile:
    regfile.write("# Region file format: DS9 version 4.1\n")
    regfile.write("global color=green dashlist=8 3 width=1 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n")
    regfile.write("fk5\n")

    # over table'objects, uncomment for required age bin
    for obj in table:
        RA, Dec, log_age = obj['RA'], obj['Dec'], obj['Age log(t)'] #check whether the column names are as it is in the data sheet
        #if log_age>0 and log_age<7:
        # write.reg（use pixel coor）
            #regfile.write('circle({},{},1.0") #color=magenta\n'.format(RA,Dec))
        
        #if log_age>=7 and log_age<8:
            #regfile.write('circle({},{},1.0") #color=blue\n'.format(RA,Dec))
        
        #if log_age>=8 and log_age<9:
            #regfile.write('circle({},{},1.0") #color=green\n'.format(RA,Dec))
            
        #if log_age>=9 and log_age<9.7:
            #regfile.write('circle({},{},1.0") #color=yellow\n'.format(RA,Dec))
            
        #if log_age>=9.7:
            #regfile.write('circle({},{},1.0") #color=red\n'.format(RA,Dec))
        
        else:
            continue

Created on 11/09/2024

Author - Ragavee Mekala

Description - This code computes the resonance locations of star clusters of the galaxy. This returns the corotation radius, Inner and Outer Lindblad resonance points and Lindblad 4:1 resonace point. The radius can be obtained by r=3600*atan(output/distance) in arcseconds.

In [ ]:
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import numpy as np
import math

#obtain pattern speed in km /s /kpc
omega_pattern=float(input('Enter the pattern speed of the galaxy: ')) # example input : 45.60
#take data from rotation curve
radius=np.array(input('Enter the radius values in the rotation curve as a list: ')) #in kpc example input : [1,2,3,4,5,6,7,8,9,10,11,12,13]
velocity=np.array(input('Enter the corresponding velocity values from the rotation curve as a list: ')) #in km/s example input : [210,210,230,210,210,210,210,210,190,170,150,130,120]

#interpolate and model the rotation curve
r_v_spline=interp1d(radius,velocity,'cubic')


#obtain points for radius and velocity
Radius=np.linspace(radius.min(),radius.max(),100000)
V=r_v_spline(Radius)
plt.plot(Radius,V)
plt.xlabel('Radius in kpc')
plt.ylabel('Velocity in km/s')
plt.title(input('Enter the title for the rotation curve: ')) # example input : Rotation curve of NGC 5194
plt.ylim(0,250)
plt.grid()
plt.show()


#calculate angular velocity for at each radii
omega=np.array(V/Radius)

#calculate gradient of angular velocity wrt radius at each radii
d_omega_dr=np.gradient(omega,Radius)


#calculate epicyclic frequency at each radii
k=(abs(4*omega**2*(1+Radius/(2*omega)*d_omega_dr)))**(1/2)


plt.ylim(0,100)
plt.plot(Radius,omega,color='blue',label='Angular velocity')
plt.plot(Radius,omega-k/2,color='green',label='Angular velocity-k/2')
plt.plot(Radius,omega+k/2,color='red',label='Angular velocity+k/2')
plt.plot(Radius,omega-k/4,color='orange',label='Angular velocity-k/4')
plt.hlines(omega_pattern,0,Radius.max(),color='black',linestyle='--')
plt.xlabel('Radius in kpc')
plt.ylabel('Angular velocities in km/s/kpc')
plt.grid()
plt.legend()
plt.show()


#calculate angular velocities at resonance points
angular_velocity_ilr=omega_pattern+k/2
angular_velocity_olr=omega_pattern-k/2
angular_velocity_4_1=omega_pattern+k/4
angular_velocity_cr=omega_pattern

difference_cr=(abs(omega-angular_velocity_cr))
min_diff_index_cr=np.argmin(difference_cr)
print('Corotation radius (kpc):',Radius[min_diff_index_cr])

difference_ilr=(abs(omega-angular_velocity_ilr))
min_diff_index_ilr=np.argmin(difference_ilr)
print('Inner Lindblad Resonance (kpc):',Radius[min_diff_index_ilr])

difference_olr=(abs(omega-angular_velocity_olr))
min_diff_index_olr=np.argmin(difference_olr)
print('Outer Lindblad Resonance (kpc):',Radius[min_diff_index_olr])

difference_4_1=(abs(omega-angular_velocity_4_1))
min_diff_index_4_1=np.argmin(difference_4_1)
print('4:1 Resonance (kpc):',Radius[min_diff_index_4_1])

Created on 22/08/2024

Author - Ragavee Mekala

Description - This code computes the azimuthal offsets of star clusters of 
the galaxy. It returns histogram plots, box plots and kernel density estimation 
plots of the azimuthal offsets. Finally this code also computes the behaviour 
of age gradient inside and outside of the corotation radius

In [ ]:
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns
import statistics as st
from scipy.optimize import brentq
from numpy.fft import *
from scipy.stats import gaussian_kde
from unidip import UniDip
import unidip.dip as dip
import pandas as pd
from astropy.table import Table


#The bandwidth estimation is obtained based on a source code from github 
#https://github.com/tomicapretto/density_estimation/blob/master/src/density.py

def dotplot(input_x,**args):
    #Count how many times does each value occur
    unique_values,counts=np.unique(input_x, return_counts=True)
    
    #Convert 1D input into 2D array
    scatter_x=[] # x values
    scatter_y=[] #corresponding y values
    
    for idx,value in enumerate(unique_values):
        for counter in range(1,counts[idx]+1):
            scatter_x.append(value)
            scatter_y.append(counter)
        
    #draw dot plot using scatter()
    plt.scatter(scatter_x,scatter_y,s=1,**args)
    

            
            
def histogram(data, bins, range_hist=None):
    """Conditionally jitted histogram.

    Parameters
    ----------
    data : array-like
        Input data. Passed as first positional argument to ``np.histogram``.
    bins : int or array-like
        Passed as keyword argument ``bins`` to ``np.histogram``.
    range_hist : (float, float), optional
        Passed as keyword argument ``range`` to ``np.histogram``.

    Returns
    -------
    hist : array
        The number of counts per bin.
    density : array
        The density corresponding to each bin.
    bin_edges : array
        The edges of the bins used.
    """
    hist, bin_edges = np.histogram(data, bins=bins, range=range_hist)
    hist_dens = hist / (hist.sum()*np.diff(bin_edges))
    return hist, hist_dens, bin_edges



def _dct1d(x):
    """Discrete Cosine Transform in 1 Dimension.

    Parameters
    ----------
    x : numpy array
        1 dimensional array of values for which the
        DCT is desired

    Returns
    -------
    output : DTC transformed values
    """
    x_len = len(x)

    even_increasing = np.arange(0,x_len,2)
    odd_decreasing = np.arange(x_len-1,0,-2)

    x = np.concatenate((x[even_increasing], x[odd_decreasing]))

    w_1k = np.r_[1,(2*np.exp(-(0+1j)*(np.arange(1, x_len))*np.pi/(2*x_len)))]
    output = np.real(w_1k*fft(x))

    return output



def _fixed_point(t, N, k_sq, a_sq):
    """Calculate t-zeta*gamma^[l](t).

    Implementation of the function t-zeta*gamma^[l](t) derived from equation 
    (30) in [1].

    References
    ----------
    .. [1] Kernel density estimation via diffusion.
       Z. I. Botev, J. F. Grotowski, and D. P. Kroese.
       Ann. Statist. 38 (2010), no. 5, 2916--2957.
    """
    k_sq = np.asarray(k_sq, dtype=np.float64)
    a_sq = np.asarray(a_sq, dtype=np.float64)

    l = 7
    f = np.sum(np.power(k_sq, l)*a_sq*np.exp(-k_sq*np.pi**2*t))
    f *= 0.5*np.pi**(2.0*l)

    for j in np.arange(l-1,2-1,-1):
        c1 = (1+0.5**(j+0.5))/3
        c2 = np.prod(np.arange(1.0,2*j+1,2,dtype=np.float64))
        c2 /= (np.pi/2)**0.5
        t_j = np.power((c1*(c2/(N*f))),(2.0/(3.0+2.0*j)))
        f = np.sum(k_sq**j*a_sq*np.exp(-k_sq*np.pi**2.0*t_j))
        f *= 0.5*np.pi**(2*j)

    out = t-(2*N*np.pi**0.5*f)**(-0.4)
    return out



def _bw_silverman(x, x_std=None, **kwargs):  # pylint: disable=unused-argument
    """Silverman's Rule."""
    if x_std is None:
        x_std = np.std(x)
    q75, q25 = np.percentile(x,[75, 25])
    x_iqr = q75-q25
    a = min(x_std,x_iqr/1.34)
    bw = 0.9*a*len(x)**(-0.2)
    return bw



def _root(function, N, args, x):
    # The right bound is at most 0.01
    found = False
    N = max(min(1050,N),50)
    tol = 10e-12+0.01*(N - 50)/1000

    while not found:
        try:
            bw, res = brentq(function,0,0.01,args=args,full_output=True, 
                             disp=False)
            found = res.converged
        except ValueError:
            bw = 0
            tol *= 2.0
            found = False
        if bw <= 0 or tol >= 1:
            bw = (_bw_silverman(x)/np.ptp(x))**2
            return bw
    return bw




def _bw_isj(x, grid_counts=None, x_std=None, x_range=None):
    """Improved Sheather-Jones bandwidth estimation.

    Improved Sheather and Jones method as explained in [1]_. This method is 
    used internally by the KDE estimator, resulting in saved computation time 
    as minima, maxima and the grid are pre-computed.

    References
    ----------
    .. [1] Kernel density estimation via diffusion.
       Z. I. Botev, J. F. Grotowski, and D. P. Kroese.
       Ann. Statist. 38 (2010), no. 5, 2916--2957.
    """
    x_len = len(x)
    if x_range is None:
        x_min = np.min(x)
        x_max = np.max(x)
        x_range = x_max-x_min

    # Relative frequency per bin
    if grid_counts is None:
        x_std = np.std(x)
        grid_len = 256
        grid_min = x_min-0.5*x_std
        grid_max = x_max+0.5*x_std
        grid_counts, _, _ = histogram(x,grid_len,(grid_min, grid_max))
    else:
        grid_len = len(grid_counts)-1

    grid_relfreq = grid_counts/x_len

    # Discrete cosine transform of the data
    a_k = _dct1d(grid_relfreq)

    k_sq = np.arange(1, grid_len)**2
    a_sq = a_k[range(1, grid_len)]**2

    t = _root(_fixed_point, x_len, args=(x_len, k_sq, a_sq), x=x)
    h = t**0.5*x_range
    return h

def getExtremePoints(data, typeOfExtreme = None, maxPoints = None):
    """
    This method returns the indeces where there is a change in the trend of the 
    input series. typeOfExtreme = None returns all extreme points, max only 
    maximum values and min only min,
    """
    a = np.diff(data)
    asign = np.sign(a)
    signchange = ((np.roll(asign,1)-asign)!= 0).astype(int)
    idx = np.where(signchange ==1)[0]
    if typeOfExtreme == 'max' and data[idx[0]] < data[idx[1]]:
        idx = idx[1:][::2]
        
    elif typeOfExtreme == 'min' and data[idx[0]] > data[idx[1]]:
        idx = idx[1:][::2]
    elif typeOfExtreme is not None:
        idx = idx[::2]
    
    # sort ids by min value
    if 0 in idx:
        idx = np.delete(idx,0)
    if (len(data)-1) in idx:
        idx = np.delete(idx, len(data)-1)
    idx = idx[np.argsort(data[idx])]
    # If we have maxpoints we want to make sure the timeseries has a cutpoint
    # in each segment, not all on a small interval
    if maxPoints is not None:
        idx= idx[:maxPoints]
        if len(idx)<maxPoints:
            return (np.arange(maxPoints)+1)*(len(data)//(maxPoints+1))
    
    return idx

pi=math.pi

# define the corotation radius. 
#The same analysis can be performed for the inner Lindblad resonance by entering the corresponding value.
corotation_radius=float(input('Enter the corotation radius: ')) # example input : 125.816


#empty lists created
Age=[]
deviation=[]
R=[]

'''----------------------------------- start calculations ---------------------------------------------------------------'''

pitch_N=float(input('Enter the pitch angle of the Northern arm: ')) #example input: -11.7339
pitch_S=float(input('Enter the pitch angle of the Southern arm: ')) #example input: -11.2182

#reading data from csv files
with open(cluster_data, newline='') as csvfile:
    reader=csv.DictReader(csvfile)
    for row in reader:
        if row['r']=='':
            continue
        r=row['r']
        r=float(r)
        R.append(r)
        age=row['Age']
        Age.append(age)
        theta_cluster=row['theta(deg)']
    
        
        
        theta_arm_N=math.degrees((math.log(r/485))/(math.tan(math.radians(pitch_N))))
       
        if theta_arm_N>=360:
            while theta_arm_N>=360:
                theta_arm_N=theta_arm_N-360
        elif theta_arm_N<=-360:
            while theta_arm_N<=-360:
                theta_arm_N=theta_arm_N+360
        
    
        theta_arm_S=math.degrees((math.log(r/265))/(math.tan(math.radians(pitch_S))))
        
        if theta_arm_S>=360:
            while theta_arm_S>=360:
                theta_arm_S=theta_arm_S-360
        elif theta_arm_S<=-360:
            while theta_arm_S<=-360:
                theta_arm_S=theta_arm_S+360
        
        #print(theta_arm_N,theta_arm_S) 
        deviation_N=float(theta_cluster)-theta_arm_N
        
        
        if deviation_N>180:
            deviation_N=deviation_N-360
        elif deviation_N<-180:
            deviation_N=deviation_N+360
        
            
            
        deviation_S=float(theta_cluster)-theta_arm_S
        
        if deviation_S>180:
            deviation_S=deviation_S-360
        elif deviation_S<-180:
            deviation_S=deviation_S+360
            
           
        #computing minimum deviation considering both arms
        if abs(deviation_N)>=abs(deviation_S):
            min_deviation=math.radians(deviation_S)
        elif abs(deviation_S)>=abs(deviation_N):
            min_deviation=math.radians(deviation_N)

        
        if min_deviation>=0 and abs(min_deviation)<pi/2:
            deviation.append(min_deviation)
        elif min_deviation>=0 and abs(min_deviation)>pi/2 and abs(min_deviation)<pi:
            deviation.append(min_deviation-pi)
        elif min_deviation<0 and abs(min_deviation)<pi/2:
            deviation.append(min_deviation)
        elif min_deviation<0 and abs(min_deviation)>pi/2 and abs(min_deviation)<pi:
            deviation.append(min_deviation+pi)
        
'''---------------------------------------end ofcalculations-----------------------------------------------------------------'''
        
    
'''-------------------------------------------------start plotting-------------------------------------------------------'''


#seperating age-bins
#1. 0<log age<7
Age_0_to_7=[]
deviation_0_to_7=[]
radius_0_to_7=[]
deviation_0_to_7_out=[]
deviation_0_to_7_in=[]


for i in range(0,len(Age)):
    
    if float(Age[i])>0 and float(Age[i])<7:
        Age_0_to_7.append(float(Age[i]))
        deviation_0_to_7.append(deviation[i])
        radius_0_to_7.append(R[i])

#seperating age bins with corotation radius
for i in range(0,len(Age_0_to_7)):
    
    if radius_0_to_7[i]>corotation_radius:
        deviation_0_to_7_out.append(deviation_0_to_7[i])
        
        
    elif radius_0_to_7[i]<corotation_radius:
        deviation_0_to_7_in.append(deviation_0_to_7[i])
        
    else:
        continue
        
        
#print(deviation_0_to_7)
fig1=plt.figure()
ax1=fig1.add_subplot(1,1,1)
ax1.hist(deviation_0_to_7,bins=20,color='magenta')
ax1.set_xlabel('Azimuthal distance in rad')
ax1.set_ylabel('Number of clusters')
fig1.suptitle('0-10Myr')
plt.grid() 
plt.show()


#2. 7<log age<8
Age_7_to_8=[]
deviation_7_to_8=[]
radius_7_to_8=[]
deviation_7_to_8_out=[]
deviation_7_to_8_in=[]

for i in range(0,len(Age)):
    
    if float(Age[i])>=7 and float(Age[i])<8:
        Age_7_to_8.append(float(Age[i]))
        deviation_7_to_8.append(deviation[i])
        radius_7_to_8.append(R[i])
        
#seperating age bins with corotation radius
for i in range(0,len(Age_7_to_8)):
    
    if radius_7_to_8[i]>corotation_radius:
        deviation_7_to_8_out.append(deviation_7_to_8[i])
        
    elif radius_7_to_8[i]<corotation_radius:
        deviation_7_to_8_in.append(deviation_7_to_8[i])
        
    else:
        continue
        
        

#print(deviation_7_to_8)
fig2=plt.figure()
ax2=fig2.add_subplot(1,1,1)
ax2.hist(deviation_7_to_8,bins=20,color='blue')
ax2.set_xlabel('Azimuthal distance in rad')
ax2.set_ylabel('Number of clusters')
fig2.suptitle('10-100Myr')
plt.grid() 
plt.show()

#3. 8<log age<9
Age_8_to_9=[]
deviation_8_to_9=[]
radius_8_to_9=[]
deviation_8_to_9_out=[]
deviation_8_to_9_in=[]

for i in range(0,len(Age)):
    
    if float(Age[i])>=8 and float(Age[i])<9:
        Age_8_to_9.append(float(Age[i]))
        deviation_8_to_9.append(deviation[i])
        radius_8_to_9.append(R[i])
        
#seperating age bins with corotation radius
for i in range(0,len(Age_8_to_9)):
    
    if radius_8_to_9[i]>corotation_radius:
        deviation_8_to_9_out.append(deviation_8_to_9[i])
        
    elif radius_8_to_9[i]<corotation_radius:
        deviation_8_to_9_in.append(deviation_8_to_9[i])
        
    else:
        continue
        
        

#print(deviation_8_to_9)
fig3=plt.figure()
ax3=fig3.add_subplot(1,1,1)
ax3.hist(deviation_8_to_9,bins=20,color='green')
ax3.set_xlabel('Azimuthal distance in rad')
ax3.set_ylabel('Number of clusters')
fig3.suptitle('0.1-1Gyr')
plt.grid() 
plt.show()

#4. 9<log age<9.7
Age_9_to_9_7=[]
deviation_9_to_9_7=[]
radius_9_to_9_7=[]
deviation_9_to_9_7_out=[]
deviation_9_to_9_7_in=[]

for i in range(0,len(Age)):
    
    if float(Age[i])>=9 and float(Age[i])<9.7:
        Age_9_to_9_7.append(float(Age[i]))
        deviation_9_to_9_7.append(deviation[i])
        radius_9_to_9_7.append(R[i])
        
#seperating age bins with corotation radius
for i in range(0,len(Age_9_to_9_7)):
    
    if radius_9_to_9_7[i]>corotation_radius:
        deviation_9_to_9_7_out.append(deviation_9_to_9_7[i])
        
    elif radius_9_to_9_7[i]<corotation_radius:
        deviation_9_to_9_7_in.append(deviation_9_to_9_7[i])
        
    else:
        continue

        
#print(deviation_9_to_9_7)
fig4=plt.figure()
ax4=fig4.add_subplot(1,1,1)
ax4.hist(deviation_9_to_9_7,bins=20,color='yellow')
ax4.set_xlabel('Azimuthal distance in rad')
ax4.set_ylabel('Number of clusters')
fig4.suptitle('1-5Gyr')
plt.grid() 
plt.show()


#5. 9.7<log age
Age_9_7=[]
deviation_9_7=[]
radius_9_7=[]
deviation_9_7_out=[]
deviation_9_7_in=[]

for i in range(0,len(Age)):
    
    if float(Age[i])>=9.7:
        Age_9_7.append(float(Age[i]))
        deviation_9_7.append(deviation[i])
        radius_9_7.append(R[i])
        
#seperating age bins with corotation radius
for i in range(0,len(Age_9_7)):
    
    if radius_9_7[i]>corotation_radius:
        deviation_9_7_out.append(deviation_9_7[i])
        
    elif radius_9_7[i]<corotation_radius:
        deviation_9_7_in.append(deviation_9_7[i])
        
    else:
        continue
        
        

#print(deviation_9_7)
fig5=plt.figure()
ax5=fig5.add_subplot(1,1,1)
ax5.hist(deviation_9_7,bins=20,color='red')
ax5.set_xlabel('Azimuthal distance in rad')
ax5.set_ylabel('Number of clusters')
fig5.suptitle('>5Gyr');
plt.grid() 
plt.show()


'''-----------------------------------------------Box plots--------------------------------------------------------'''

'''--------------------------------for age gradients----------------------------------------------------------------'''

#plotting box plots
data=[deviation_0_to_7,deviation_7_to_8,deviation_8_to_9,deviation_9_to_9_7,deviation_9_7]
fig=plt.figure()
ax=fig.add_subplot(1,1,1)
plt.boxplot(data)
plt.xticks(rotation=40)
ax.set_xlabel('Age')
ax.set_ylabel('Azimuthal offsets (rad)')
plt.grid()
fig.suptitle('Box plot showing age gradient')

#connecting the median values
X=[]
Y=[]
for m in plt.boxplot(data)['medians']:
    m.set_color('black')
    [[x0,x1],[y0,y1]]=m.get_data()
    X.append(np.mean((x0,x1)))
    Y.append(np.mean((y0,y1)))
plt.plot(X,Y,c='blue')
plt.xticks([1,2,3,4,5],['0-10Myr','10-100Myr','0.1-1Gyr','1-5Gyr','>5Gyr'])    
plt.show()

'''---------------------------------------end of box plots--------------------------------------------------------'''

'''--------------------------------------KDE PLOTS---------------------------------------------------------------'''


#kdeplots

#estimating bandwidths
bandwidth1=_bw_isj(deviation_0_to_7)
bandwidth2=_bw_isj(deviation_7_to_8)
bandwidth3=_bw_isj(deviation_8_to_9)
bandwidth4=_bw_isj(deviation_9_to_9_7)
bandwidth5=_bw_isj(deviation_9_7)
#print([bandwidth1,bandwidth2,bandwidth3,bandwidth4,bandwidth5])
bandwidth=max([bandwidth1,bandwidth2,bandwidth3,bandwidth4,bandwidth5,0.5])
print(bandwidth)

#kdeplots
plot1=sns.kdeplot(deviation_0_to_7,bw_adjust=bandwidth,color='magenta',kernel='gaussian',shade=True)
plot2=sns.kdeplot(deviation_7_to_8,bw_adjust=bandwidth,color='blue',kernel='gaussian',shade=True)
plot3=sns.kdeplot(deviation_8_to_9,bw_adjust=bandwidth,color='green',kernel='gaussian',shade=True)
plot4=sns.kdeplot(deviation_9_to_9_7,bw_adjust=bandwidth,color='yellow',kernel='gaussian',shade=True)
plot5=sns.kdeplot(deviation_9_7,bw_adjust=bandwidth,color='red',kernel='gaussian',shade=True)


'''----------------------------------------FWHM 0-10Myr-------------------------------------------------------'''

#plotting maximum of age bin 0-10Myr
x1,y1=sns.kdeplot(deviation_0_to_7,bw_adjust=bandwidth,color='magenta',label='0-10Myr',).lines[0].get_data()
idx1=getExtremePoints(y1,typeOfExtreme='max')
i=1
for i in idx1:
    plt.vlines(x=x1[i],ymin=0,ymax=y1[i],color='magenta',linestyle='--')
    #print(x1[i],y1[i])
    
#plotting FWHM
maxpos = y1.argmax()
halfmax=y1.max()/2
y1_left=y1[:maxpos]
deviation_left=np.abs(y1[:maxpos]-halfmax)
deviation_l_sorted=np.sort(deviation_left)


for deviation in deviation_left:
    
    if deviation==deviation_l_sorted[1]:            
        left_pos=np.where(deviation_left==deviation_l_sorted[0]) #choose indices manually to check whether FWHM is plotted by uncommenting the horizontal line
        #print(x1[left_pos])


y1_right=y1[maxpos:]
deviation_right=np.abs(y1[maxpos:]-halfmax)
deviation_r_sorted=np.sort(deviation_right)

for deviation in deviation_right:
    if deviation==deviation_r_sorted[0]:
        right_pos=np.where(deviation_right==deviation_r_sorted[0])+maxpos
        #print(x1[right_pos])
    

fullwidthathalfmax_1 = x1[right_pos] - x1[left_pos]
#print(fullwidthathalfmax_1)
#plt.hlines(halfmax, x1[left_pos], x1[right_pos], color='black', ls=':')



'''-------------------------------------------FWHM 10-100Myr----------------------------------------------------'''

#plotting maximum of age bin 10-100Myr    
x2,y2=sns.kdeplot(deviation_7_to_8,bw_adjust=bandwidth,color='blue',label='10-100Myr').lines[1].get_data()
idx2=getExtremePoints(y2,typeOfExtreme='max')
i=1
for i in idx2:
    plt.vlines(x=x2[i],ymin=0,ymax=y2[i],color='blue',linestyle='--')
    #print(x2[i],y2[i])
    
#plotting FWHM
maxpos = y2.argmax()
halfmax=y2.max()/2
y2_left=y2[:maxpos]
deviation_left=np.abs(y2[:maxpos]-halfmax)
deviation_l_sorted=np.sort(deviation_left)


for deviation in deviation_left:
    if deviation==deviation_l_sorted[0]:
        left_pos=np.where(deviation_left==deviation_l_sorted[0])
        #print(x2[left_pos])




y2_right=y2[maxpos:]
deviation_right=np.abs(y2[maxpos:]-halfmax)
deviation_r_sorted=np.sort(deviation_right)

for deviation in deviation_right:
    if deviation==deviation_r_sorted[0]:
        right_pos=np.where(deviation_right==deviation_r_sorted[0])+maxpos
        #print(x2[right_pos])
            
   
fullwidthathalfmax_2 = x2[right_pos] - x2[left_pos]
#print(fullwidthathalfmax_2)
#plt.hlines(halfmax, x2[left_pos], x2[right_pos], color='black', ls=':') 


'''-------------------------------------------FWHM 0.1-1Gyr----------------------------------------------------'''
    
#plotting maximum of age bin 0.1-1Gyr   
x3,y3=sns.kdeplot(deviation_8_to_9,bw_adjust=bandwidth,color='green',label='0.1-1Gyr').lines[2].get_data()
idx3=getExtremePoints(y3,typeOfExtreme='max')
i=1
for i in idx3:
    plt.vlines(x=x3[i],ymin=0,ymax=y3[i],color='green',linestyle='--')
    #print(x3[i],y3[i])
    
#plotting FWHM
maxpos = y3.argmax()
halfmax=y3.max()/2
y3_left=y3[:maxpos]
deviation_left=np.abs(y3[:maxpos]-halfmax)
deviation_l_sorted=np.sort(deviation_left)


for deviation in deviation_left:
    if deviation==deviation_l_sorted[0]:
        left_pos=np.where(deviation_left==deviation_l_sorted[0]) 
        #print(x3[left_pos])
            
   
y3_right=y3[maxpos:]
deviation_right=np.abs(y3[maxpos:]-halfmax)
deviation_r_sorted=np.sort(deviation_right)

for deviation in deviation_right:
    if deviation==deviation_r_sorted[0]:
        right_pos=np.where(deviation_right==deviation_r_sorted[0])+maxpos
        #print(x3[right_pos])
            

fullwidthathalfmax_3 = x3[right_pos] - x3[left_pos]
#print(fullwidthathalfmax_3)
#plt.hlines(halfmax, x3[left_pos], x3[right_pos], color='black', ls=':')       



'''---------------------------------------------FWHM 1-5 Gyr--------------------------------------------------'''

#plotting maximum of age bin 1-5Gyr
x4,y4=sns.kdeplot(deviation_9_to_9_7,bw_adjust=bandwidth,color='yellow',label='1-5Gyr').lines[3].get_data()
idx4=getExtremePoints(y4,typeOfExtreme='max')
i=1
for i in idx4:
    plt.vlines(x=x4[i],ymin=0,ymax=y4[i],color='yellow',linestyle='--')
    #print(x4[i],y4[i])

#plotting FWHM
maxpos = y4.argmax()
halfmax=y4.max()/2
y4_left=y4[:maxpos]
deviation_left=np.abs(y4[:maxpos]-halfmax)
deviation_l_sorted=np.sort(deviation_left)


for deviation in deviation_left:
    if deviation==deviation_l_sorted[0]:
        left_pos=np.where(deviation_left==deviation_l_sorted[0])
        #print(x4[left_pos])
    
    
y4_right=y4[maxpos:]
deviation_right=np.abs(y4[maxpos:]-halfmax)
deviation_r_sorted=np.sort(deviation_right)

for deviation in deviation_right:
   if deviation==deviation_r_sorted[0]:
        right_pos=np.where(deviation_right==deviation_r_sorted[0])+maxpos
        #print(x4[right_pos])
        


fullwidthathalfmax_4 = x4[right_pos] - x4[left_pos]
#print(fullwidthathalfmax_4)
#plt.hlines(halfmax, x4[left_pos], x4[right_pos], color='black', ls=':') 


'''-------------------------------------------FWHM >5Gyr----------------------------------------------------'''


#plotting maximum of age bin >5Gyr
x5,y5=sns.kdeplot(deviation_9_7,bw_adjust=bandwidth,color='red',label='>5Gyr').lines[4].get_data()
idx5=getExtremePoints(y5,typeOfExtreme='max')
i=1
for i in idx5:
    plt.vlines(x=x5[i],ymin=0,ymax=y5[i],color='red',linestyle='--')
    print(x5[i],y5[i])

#plotting FWHM
maxpos = y5.argmax()
halfmax=y5.max()/2
y5_left=y5[:maxpos]
deviation_left=np.abs(y5[:maxpos]-halfmax)
deviation_l_sorted=np.sort(deviation_left)


for deviation in deviation_left:
    if deviation==deviation_l_sorted[0]:
        left_pos=np.where(deviation_left==deviation_l_sorted[0])
        #print(x5[left_pos])
    
    
y5_right=y5[maxpos:]
deviation_right=np.abs(y5[maxpos:]-halfmax)
deviation_r_sorted=np.sort(deviation_right)

for deviation in deviation_right:
   if deviation==deviation_r_sorted[0]:
        right_pos=np.where(deviation_right==deviation_r_sorted[0])+maxpos
        #print(x5[right_pos])
        


fullwidthathalfmax_5 = x5[right_pos] - x5[left_pos]
#print(fullwidthathalfmax_5)
#plt.hlines(halfmax, x5[left_pos], x5[right_pos], color='black', ls=':') 

'''----------------------------------- labelling the axes -----------------------------------------------------------'''


plt.legend(loc='upper right')
plt.xlabel('Azimuthal offsets (rad)')
plt.ylabel('Kernel density estimate')
plt.title('Kernel Density Estimates of Azimuthal Offsets')
plt.grid()
plt.show()


'''---------------------------------------Cumulative KDEs----------------------------------------------------'''

#plotting cumulative KDEs
plot6=sns.kdeplot(deviation_0_to_7,bw_adjust=bandwidth,color='magenta',kernel='gaussian',shade=True,cumulative=True,label='0-10Myr')
plot7=sns.kdeplot(deviation_7_to_8,bw_adjust=bandwidth,color='blue',kernel='gaussian',shade=True,cumulative=True,label='10-100Myr')
plot8=sns.kdeplot(deviation_8_to_9,bw_adjust=bandwidth,color='green',kernel='gaussian',shade=True,cumulative=True,label='0.1-1Gyr')
plot9=sns.kdeplot(deviation_9_to_9_7,bw_adjust=bandwidth,color='yellow',kernel='gaussian',shade=True,cumulative=True,label='1-5Gyr')
plot10=sns.kdeplot(deviation_9_7,bw_adjust=bandwidth,color='red',kernel='gaussian',shade=True,cumulative=True,label='>5Gyr')

plt.title('Cumulative Kernel Density Estimates of Azimuthal Offsets')
plt.xlabel('Azimuthal offsets (rad)')
plt.ylabel('Kernel density estimation (cumulative)')
plt.legend(loc='upper left')
plt.grid()
plt.show()
    

'''------------------------------------Plotting peaks, medians, fwhms and mean values---------------------------------'''

plt.figure(figsize=(6,4),dpi=200)
x_axis=['0-10Myr','10-100Myr','0.1-1Gyr','1-5Gyr','>5Gyr']
y_axis_1=[st.mean(deviation_0_to_7),st.mean(deviation_7_to_8),st.mean(deviation_8_to_9),st.mean(deviation_9_to_9_7),st.mean(deviation_9_7)]
y_axis_2=[st.median(deviation_0_to_7),st.median(deviation_7_to_8),st.median(deviation_8_to_9),st.median(deviation_9_to_9_7),st.median(deviation_9_7)]
y_axis_3=[x1[y1.argmax()],x2[y2.argmax()],x3[y3.argmax()],x4[y4.argmax()],x5[y5.argmax()]]


plt.plot(x_axis,y_axis_1,'.',color='blue',label='Mean deviation',markersize=5)
plt.plot(x_axis,y_axis_2,'.',color='green',label='Median deviation',markersize=5)
plt.plot(x_axis,y_axis_3,'.',color='red',label='Peak',markersize=5)
plt.plot(x_axis,y_axis_1,linewidth=1,color='blue')
plt.plot(x_axis,y_axis_2,linewidth=1,color='green')
plt.plot(x_axis,y_axis_3,linewidth=1,color='red')

plt.legend(loc='upper right')
plt.xlabel('Age bins')
plt.ylabel('Azimuthal offsets (rad)')
plt.title('Statistics of stellar cluster distribution')
plt.grid()
plt.show()



In [ ]:
'''-----------------------------------------Deviations of age bins inside and outside corotation radius-----------------'''


#1. 0<log age<7
#inside CR
fig1_in=plt.figure()
ax1_in=fig1_in.add_subplot(1,1,1)
ax1_in.hist(deviation_0_to_7_in,bins=20,color='magenta')
ax1_in.set_xlabel('Azimuthal distance in rad')
ax1_in.set_ylabel('Number of clusters')
fig1_in.suptitle('0-10Myr - inside corotation radius')
plt.grid() 
plt.show()

#outside CR
fig1_out=plt.figure()
ax1_out=fig1_out.add_subplot(1,1,1)
ax1_out.hist(deviation_0_to_7_out,bins=20,color='magenta')
ax1_out.set_xlabel('Azimuthal distance in rad')
ax1_out.set_ylabel('Number of clusters')
fig1_out.suptitle('0-10Myr - outside corotation radius')
plt.grid()
plt.show()

#2. 7<log age<8
#inside CR
fig2_in=plt.figure()
ax2_in=fig2_in.add_subplot(1,1,1)
ax2_in.hist(deviation_7_to_8_in,bins=20,color='blue')
ax2_in.set_xlabel('Azimuthal distance in rad')
ax2_in.set_ylabel('Number of clusters')
fig2_in.suptitle('10-100Myr - inside corotation radius')
plt.grid() 
plt.show()

#outside CR
fig2_out=plt.figure()
ax2_out=fig2_out.add_subplot(1,1,1)
ax2_out.hist(deviation_7_to_8_out,bins=20,color='blue')
ax2_out.set_xlabel('Azimuthal distance in rad')
ax2_out.set_ylabel('Number of clusters')
fig2_out.suptitle('0-10Myr - outside corotation radius')
plt.grid()
plt.show()

#3. 8<log age<9
#inside CR
fig3_in=plt.figure()
ax3_in=fig3_in.add_subplot(1,1,1)
ax3_in.hist(deviation_8_to_9_in,bins=20,color='green')
ax3_in.set_xlabel('Azimuthal distance in rad')
ax3_in.set_ylabel('Number of clusters')
fig3_in.suptitle('0.1-1Gyr - inside corotation radius')
plt.grid() 
plt.show()

#outside CR
fig3_out=plt.figure()
ax3_out=fig3_out.add_subplot(1,1,1)
ax3_out.hist(deviation_8_to_9_out,bins=20,color='green')
ax3_out.set_xlabel('Azimuthal distance in rad')
ax3_out.set_ylabel('Number of clusters')
fig3_out.suptitle('0.1-1Gyr - outside corotation radius')
plt.grid()
plt.show()


#4. 9<log age<9.7
#inside CR
fig4_in=plt.figure()
ax4_in=fig4_in.add_subplot(1,1,1)
ax4_in.hist(deviation_9_to_9_7_in,bins=20,color='yellow')
ax4_in.set_xlabel('Azimuthal distance in rad')
ax4_in.set_ylabel('Number of clusters')
fig4_in.suptitle('1-5Gyr - inside corotation radius')
plt.grid() 
plt.show()

#outside CR
fig4_out=plt.figure()
ax4_out=fig4_out.add_subplot(1,1,1)
ax4_out.hist(deviation_9_to_9_7_out,bins=20,color='yellow')
ax4_out.set_xlabel('Azimuthal distance in rad')
ax4_out.set_ylabel('Number of clusters')
fig4_out.suptitle('1-5Gyr - outside corotation radius')
plt.grid()
plt.show()

#5. 9.7<log age
#inside CR
fig5_in=plt.figure()
ax5_in=fig5_in.add_subplot(1,1,1)
ax5_in.hist(deviation_9_7_in,bins=20,color='red')
ax5_in.set_xlabel('Azimuthal distance in rad')
ax5_in.set_ylabel('Number of clusters')
fig4_in.suptitle('>5Gyr - inside corotation radius')
plt.grid() 
plt.show()

#outside CR
fig5_out=plt.figure()
ax5_out=fig5_out.add_subplot(1,1,1)
ax5_out.hist(deviation_9_7_out,bins=20,color='red')
ax5_out.set_xlabel('Azimuthal distance in rad')
ax5_out.set_ylabel('Number of clusters')
fig5_out.suptitle('>5Gyr - outside corotation radius')
plt.grid()
plt.show()

'''--------------------------------------------Box plots----------------------------------------------------'''



'''--------------------------------------------inside corotation radius-------------------------------------------'''

#plotting box plots
data_in=[deviation_0_to_7_in,deviation_7_to_8_in,deviation_8_to_9_in,deviation_9_to_9_7_in,deviation_9_7_in]
fig=plt.figure()
ax=fig.add_subplot(1,1,1)
plt.boxplot(data_in)
plt.xticks(rotation=40)
ax.set_xlabel('Age')
ax.set_ylabel('Azimuthal offsets (rad)')
plt.grid()
fig.suptitle('Box plot showing age gradient inside corotation radius')

#connecting the median values
X=[]
Y=[]
for m in plt.boxplot(data_in)['medians']:
    m.set_color('black')
    [[x0,x1],[y0,y1]]=m.get_data()
    X.append(np.mean((x0,x1)))
    Y.append(np.mean((y0,y1)))
plt.plot(X,Y,c='blue')
plt.xticks([1,2,3,4,5],['0-10Myr','10-100Myr','0.1-1Gyr','1-5Gyr','>5Gyr'])  
plt.show()


'''------------------------------------Plotting peaks, medians, fwhms and mean values---------------------------------'''


plt.figure(figsize=(6,4),dpi=200)
x_axis=['0-10Myr','10-100Myr','0.1-1Gyr','1-5Gyr','>5Gyr']

y_axis_1=[st.mean(deviation_0_to_7_in),st.mean(deviation_7_to_8_in),st.mean(deviation_8_to_9_in),st.mean(deviation_9_to_9_7_in),st.mean(deviation_9_7_in)]
y_axis_2=[st.median(deviation_0_to_7_in),st.median(deviation_7_to_8_in),st.median(deviation_8_to_9_in),st.median(deviation_9_to_9_7_in),st.median(deviation_9_7_in)]
y_axis_3=[st.mode(deviation_0_to_7_in),st.mode(deviation_7_to_8_in),st.mode(deviation_8_to_9_in),st.mode(deviation_9_to_9_7_in),st.mode(deviation_9_7_in)]

plt.plot(x_axis,y_axis_1,'.',color='blue',label='Mean deviation',markersize=5)
plt.plot(x_axis,y_axis_2,'.',color='green',label='Median deviation',markersize=5)
plt.plot(x_axis,y_axis_3,'.',color='red',label='Peak',markersize=5)
plt.plot(x_axis,y_axis_1,linewidth=1,color='blue')
plt.plot(x_axis,y_axis_2,linewidth=1,color='green')
plt.plot(x_axis,y_axis_3,linewidth=1,color='red')
plt.grid()

plt.legend(loc='upper right')
plt.title('Statistics of stellar cluster distribution within the corotation radius')
plt.show()
     

    
'''-----------------------------------------------outside corotation radius--------------------------------------'''

#plotting box plots
data_out=[deviation_0_to_7_out,deviation_7_to_8_out,deviation_8_to_9_out,deviation_9_to_9_7_out,deviation_9_7_out]
fig=plt.figure()
ax=fig.add_subplot(1,1,1)
plt.boxplot(data_out)
plt.xticks(rotation=40)
ax.set_xlabel('Age')
ax.set_ylabel('Azimuthal offsets (rad)')
plt.grid()
fig.suptitle('Box plot showing age gradient outside corotation radius')

#connecting the median values
X=[]
Y=[]
for m in plt.boxplot(data_out)['medians']:
    m.set_color('black')
    [[x0,x1],[y0,y1]]=m.get_data()
    X.append(np.mean((x0,x1)))
    Y.append(np.mean((y0,y1)))
plt.plot(X,Y,c='blue')
plt.xticks([1,2,3,4,5],['0-10Myr','10-100Myr','0.1-1Gyr','1-5Gyr','>5Gyr'])   
plt.show()



'''------------------------------------Plotting peaks, medians, fwhms and mean values---------------------------------'''


plt.figure(figsize=(6,4),dpi=200)
x_axis=['0-10Myr','10-100Myr','0.1-1Gyr','1-5Gyr','>5Gyr']
y_axis_1=[st.mean(deviation_0_to_7_out),st.mean(deviation_7_to_8_out),st.mean(deviation_8_to_9_out),st.mean(deviation_9_to_9_7_out),st.mean(deviation_9_7_out)]
y_axis_2=[st.median(deviation_0_to_7_out),st.median(deviation_7_to_8_out),st.median(deviation_8_to_9_out),st.median(deviation_9_to_9_7_out),st.median(deviation_9_7_out)]
y_axis_3=[st.mode(deviation_0_to_7_out),st.mode(deviation_7_to_8_out),st.mode(deviation_8_to_9_out),st.mode(deviation_9_to_9_7_out),st.mode(deviation_9_7_out)]

plt.plot(x_axis,y_axis_1,'.',color='blue',label='Mean deviation',markersize=5)
plt.plot(x_axis,y_axis_2,'.',color='green',label='Median deviation',markersize=5)
plt.plot(x_axis,y_axis_3,'.',color='red',label='Peak',markersize=5)
plt.plot(x_axis,y_axis_1,linewidth=1,color='blue')
plt.plot(x_axis,y_axis_2,linewidth=1,color='green')
plt.plot(x_axis,y_axis_3,linewidth=1,color='red')
plt.grid()
plt.title('Statistics of stellar cluster distribution beyond the corotation radius')
plt.legend(loc='upper right')
plt.show()




Created on 07/10/2024

Author - Ragavee Mekala

Description - This code verifies whether the deviations and ages are coincident with each other computed using relative angular velocity of the density wave and material present in the galaxy.

In [ ]:
pi=np.pi
velocity_matter=V
velocity_pattern=omega_pattern*Radius

#creating empty lists
time_lapse_0_to_7=[]
time_lapse_7_to_8=[]
time_lapse_8_to_9=[]


radius_0_to_7_real=radius_0_to_7/(corotation_radius/Radius[min_diff_index_cr])
#list of ages in Myr for 0-10Myr

n=0
for radius in radius_0_to_7_real:
    i=np.argmin(abs(Radius-radius))
    relative_velocity=velocity_pattern[i]-velocity_matter[i]
    rel_angular_velocity=(relative_velocity/(Radius[i]))/(150e+9/math.tan(pi/(180*3600))) #in /s
    time_lapse_0_to_7.append((deviation_0_to_7[n]/rel_angular_velocity)/(60*60*24*365*1e+6)) #in Myr
    n=n+1
 
time_lapse_0_to_7=np.round(time_lapse_0_to_7,2)
print(min(time_lapse_0_to_7),max(time_lapse_0_to_7))
print('Time lapse for 0-10Myr (in Myr):',abs(st.mean(time_lapse_0_to_7)))


radius_7_to_8_real=radius_7_to_8/(corotation_radius/Radius[min_diff_index_cr])
#list of ages in Myr for 10-100Myr

n=0
for radius in radius_7_to_8_real:
    i=np.argmin(abs(Radius-radius))
    relative_velocity=velocity_pattern[i]-velocity_matter[i]
    rel_angular_velocity=(relative_velocity/(Radius[i]))/(150e+9/math.tan(pi/(180*3600))) #in /s
    time_lapse_7_to_8.append((deviation_7_to_8[n]/rel_angular_velocity)/(60*60*24*365*1e+6)) #in Myr
    n=n+1

time_lapse_7_to_8=np.round(time_lapse_7_to_8,2)
print(min(time_lapse_7_to_8),max(time_lapse_7_to_8))
print('Time lapse for 10-100Myr (in Myr):',abs(st.mean(time_lapse_7_to_8)))


radius_8_to_9_real=radius_8_to_9/(corotation_radius/Radius[min_diff_index_cr])
#list of ages in Myr for 10-100Myr

n=0
for radius in radius_8_to_9_real:
    i=np.argmin(abs(Radius-radius))
    relative_velocity=velocity_pattern[i]-velocity_matter[i]
    rel_angular_velocity=(relative_velocity/(Radius[i]))/(150e+9/math.tan(pi/(180*3600))) #in /s
    time_lapse_8_to_9.append((deviation_8_to_9[n]/rel_angular_velocity)/(60*60*24*365*1e+6)) #in Myr
    n=n+1

time_lapse_8_to_9=np.round(time_lapse_8_to_9,2)
print(min(time_lapse_8_to_9),max(time_lapse_8_to_9))
print('Time lapse for 100-1000Myr (in Myr):',abs(st.mean(time_lapse_8_to_9)))


Created on 07/10/2024

Author - Ragavee Mekala

Description - This code predicts the deviations of young star clusters(0-10Myr) in the future and check whether they are consistent with the deviations found in the data considering old star clusters (>1Gyr) in the galaxy.

In [ ]:
radii_real=radius_0_to_7_real
deviation_9_to_9_7_future=[]


n=0
for radius in radii_real:
    i=np.argmin(abs(Radius-radius))
    relative_velocity=velocity_pattern[i]-velocity_matter[i]
    rel_angular_velocity=(relative_velocity/(Radius[i]))/(150e+9/math.tan(pi/(180*3600))) #in /s
    for age in range(100,500):
        deviation=rel_angular_velocity*60*60*24*365*1e+7*age/180*pi #print(st.mean(np.array(Age_9_to_9_7)))
        while abs(deviation)>=2*pi:
            if deviation>=2*pi:
                deviation=deviation-2*pi
            elif deviation<=-2*pi:
                deviation=deviation+2*pi

        if deviation>=0 and abs(deviation)<pi/2:
            deviation_9_to_9_7_future.append(deviation)
        elif deviation>=0 and abs(deviation)>pi/2 and abs(deviation)<pi:
            deviation_9_to_9_7_future.append(deviation-pi)
        elif deviation<0 and abs(deviation)<pi/2:
            deviation_9_to_9_7_future.append(deviation)
        elif deviation<0 and abs(deviation)>pi/2 and abs(deviation)<pi:
            deviation_9_to_9_7_future.append(deviation+pi)

deviation_9_to_9_7_array=np.array(deviation_9_to_9_7)
indices_positive=np.where((deviation_9_to_9_7_array>pi/2) & (deviation_9_to_9_7_array<pi))
indices_negative=np.where((deviation_9_to_9_7_array<-pi/2) & (deviation_9_to_9_7_array>-pi))
for indices in indices_positive:
    deviation_9_to_9_7_array[indices]=deviation_9_to_9_7_array[indices]-pi
for indices in indices_negative:
    deviation_9_to_9_7_array[indices]=deviation_9_to_9_7_array[indices]+pi



In [ ]:
#Box plot
plt.figure(figsize=(6,4),dpi=200)
plt.boxplot([deviation_9_to_9_7_array, deviation_9_to_9_7_future], labels=['Observations', 'Expected'])
plt.ylabel('Angular Deviation in rad')
plt.title('Box Plot of Observed vs. Expected Angular Deviations')
plt.grid()
plt.show()


#KDE plots
plt.figure(figsize=(6,4),dpi=200)
bandwidth_future=_bw_isj(deviation_9_to_9_7_future)
bandwidth_array=_bw_isj(deviation_9_to_9_7_array)
bandwidth=max(bandwidth_future,bandwidth_array,0.5)
plot_future=sns.kdeplot(deviation_9_to_9_7_future,bw_adjust=bandwidth,color='blue',shade=True,label='Expected')
plot_array=sns.kdeplot(deviation_9_to_9_7_array,bw_adjust=bandwidth,color='red',shade=True,label='Observed')
plt.xlabel('Angular deviation in rad')
plt.ylabel('Kernel density estimate')
plt.title('Kernel Density Estimate Plot of Observed and Expected Deviations')
plt.legend()
plt.grid()
plt.show()

#Error bar plots
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4),dpi=200)
actual_mean = np.mean(deviation_9_to_9_7_array) # Calculate means and standard deviations
predicted_mean = np.mean(deviation_9_to_9_7_future)
actual_std = np.std(deviation_9_to_9_7_array)
predicted_std = np.std(deviation_9_to_9_7_future)
plt.errorbar(actual_mean,predicted_mean,xerr=actual_std,yerr=predicted_std, fmt='ro', capsize=5, markersize=8, ecolor='blue')
plt.xlabel('Observed Angular Deviation values') # Add plot labels and title
plt.ylabel('Expected Angular Deviation values')
plt.title('Error Bar Plot: Observed vs. Expected Angular Deviations')
plt.grid()
plt.show()


Created on 07/10/2024

Author - Ragavee Mekala

Description - This code predicts the deviations of young star clusters(0-10Myr) in the future and check whether they are consistent with the deviations found in the data considering old star clusters (>10Gyr) in the galaxy.

In [ ]:
deviation_9_7_future=[]

n=0
for radius in radii_real:
    i=np.argmin(abs(Radius-radius))
    relative_velocity=velocity_pattern[i]-velocity_matter[i]
    rel_angular_velocity=(relative_velocity/(Radius[i]))/(150e+9/math.tan(pi/(180*3600))) #in /s
    for age in range(500,1300):
        deviation=rel_angular_velocity*60*60*24*365*1e+7*age/180*pi #print(st.mean(np.array(Age_9_7)))
        while abs(deviation)>=2*pi:
            if deviation>=2*pi:
                deviation=deviation-2*pi
            elif deviation<=-2*pi:
                deviation=deviation+2*pi

        if deviation>=0 and abs(deviation)<pi/2:
            deviation_9_7_future.append(deviation)
        elif deviation>=0 and abs(deviation)>pi/2 and abs(deviation)<pi:
            deviation_9_7_future.append(deviation-pi)
        elif deviation<0 and abs(deviation)<pi/2:
            deviation_9_7_future.append(deviation)
        elif deviation<0 and abs(deviation)>pi/2 and abs(deviation)<pi:
            deviation_9_7_future.append(deviation+pi)

deviation_9_7_array=np.array(deviation_9_7)
indices_positive=np.where((deviation_9_7_array>pi/2) & (deviation_9_7_array<pi))
indices_negative=np.where((deviation_9_7_array<-pi/2) & (deviation_9_7_array>-pi))
for indices in indices_positive:
    deviation_9_7_array[indices]=deviation_9_7_array[indices]-pi
for indices in indices_negative:
    deviation_9_7_array[indices]=deviation_9_7_array[indices]+pi


In [ ]:
#Box plot
plt.figure(figsize=(6,4),dpi=200)
plt.boxplot([deviation_9_7_array, deviation_9_7_future], labels=['Observations', 'Expected'])
plt.ylabel('Angular Deviation in rad')
plt.title('Box Plot of Observed vs. Expected Angular Deviations')
plt.grid()
plt.show()


#KDE plots
plt.figure(figsize=(6,4),dpi=200)
bandwidth_future=_bw_isj(deviation_9_7_future)
bandwidth_array=_bw_isj(deviation_9_7_array)
bandwidth=max(bandwidth_future,bandwidth_array)
print('The bandwidth is',bandwidth)
plot_future=sns.kdeplot(deviation_9_7_future,bw_adjust=0.5,color='blue',shade=True,label='Expected')
plot_array=sns.kdeplot(deviation_9_7_array,bw_adjust=bandwidth,color='red',shade=True,label='Observed')
plt.xlabel('Angular deviation in rad')
plt.ylabel('Kernel density estimate')
plt.title('Kernel Density Estimate Plot of Observed and Expected Deviations')
plt.legend()
plt.grid()
plt.show()

#Error bar plots
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4),dpi=200)
actual_mean = np.mean(deviation_9_7_array) # Calculate means and standard deviations
predicted_mean = np.mean(deviation_9_7_future)
actual_std = np.std(deviation_9_7_array)
predicted_std = np.std(deviation_9_7_future)
plt.errorbar(actual_mean,predicted_mean,xerr=actual_std,yerr=predicted_std, fmt='ro', capsize=5, markersize=8, ecolor='blue')
plt.xlabel('Observed Angular Deviation values') # Add plot labels and title
plt.ylabel('Expected Angular Deviation values')
plt.title('Error Bar Plot: Observed vs. Expected Angular Deviations')
plt.grid()
plt.show()


This code creates reg files to create a timelapse from 0.01Gyr to 10Gyr

In [ ]:
deviation__future=[]

X_center=float(input('Enter the center X coordinate in image: ')) #input example : 480
Y_center=float(input('Enter the center Y coordinate in image: ')) #input example : 480

n=0
for radius in radii_real:
    i=np.argmin(abs(Radius-radius))
    relative_velocity=velocity_pattern[i]-velocity_matter[i]
    rel_angular_velocity=(relative_velocity/(Radius[i]))/(150e+9/math.tan(pi/(180*3600))) #in /s
    deviation=rel_angular_velocity*60*60*24*365*1e7/180*pi #change years
    deviation__future.append(deviation)

df = pd.read_csv(cluster_data)

# CSV to Astropy Table
table = Table.from_pandas(df)
    

reg_filename=input('Enter file name for the region file for 1e7 yrs: ') # example input : NGC5194_future_1e7.reg
with open(reg_filename,'w') as regfile:
    regfile.write("# Region file format: DS9 version 4.1\n")
    regfile.write("global color=green dashlist=8 3 width=1 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n")
    regfile.write("image\n")
    #regfile.write('circle(480,480,125.816) #color=white\n') #Uncomment this to mark the corotation radius and edit the coordinates of the circle are X,Y coordinates of the center and the radius in image
    # The above coordinates are for NGC 5194
    for obj in table:
        radius,initial_angle,log_age=obj['r'],obj['theta(360)'], obj['Age']
        index=np.argmin(abs(radius_0_to_7-radius))
        initial_angle=math.radians(initial_angle)
        new_angle=initial_angle+deviation__future[index]
        X=radius_0_to_7[index]*math.cos(new_angle)+X_center
        Y=radius_0_to_7[index]*math.sin(new_angle)+Y_center
        X=np.round(X,2)
        Y=np.round(Y,2)
        if log_age>0 and log_age<7:
            regfile.write('circle({},{},1.0") #color=green\n'.format(X,Y))
        else:
            continue
            
for n in range(0,50):
    
    deviation__future=[]
    for radius in radii_real:
        i=np.argmin(abs(Radius-radius))
        relative_velocity=velocity_pattern[i]-velocity_matter[i]
        rel_angular_velocity=(relative_velocity/(Radius[i]))/(150e+9/math.tan(pi/(180*3600))) #in /s
        deviation=-rel_angular_velocity*60*60*24*365*(n+1)*20e7/180*pi #change years
        deviation__future.append(deviation)
        

    df = pd.read_csv(cluster_data)

    # CSV to Astropy Table
    table = Table.from_pandas(df)
    

    reg_filename=float(input('Enter file name for the region file for all the yrs: ')) # example input : NGC5194_future_{}e7.reg.format((n+1)*20)
    with open(reg_filename,'w') as regfile:
        regfile.write("# Region file format: DS9 version 4.1\n")
        regfile.write("global color=green dashlist=8 3 width=1 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n")
        regfile.write("image\n")
        #regfile.write('circle(480,480,125.816) #color=white\n') #center coordinates and corotation radius
        for obj in table:
            radius,initial_angle,log_age=obj['r'],obj['theta(360)'], obj['Age']
            index=np.argmin(abs(radius_0_to_7-radius))
            initial_angle=math.radians(initial_angle)
            new_angle=initial_angle+deviation__future[index]
            X=radius_0_to_7[index]*math.cos(new_angle)+X_center
            Y=radius_0_to_7[index]*math.sin(new_angle)+Y_center
            X=np.round(X,2)
            Y=np.round(Y,2)
            if log_age>0 and log_age<7:
                regfile.write('circle({},{},1.0") #color=green\n'.format(X,Y))
            else:
                continue


    
        